## Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import statsmodels.api as sm
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.tsa.arima.model import ARIMA

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Conv1D, MaxPooling1D, Flatten, Concatenate, Dense, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings('ignore')


## Data

In [2]:
df_train = pd.read_csv('data/train.csv')

df_train.sample(10)

,date,store,item,sales
829786,2015-02-22,5,46,41
726954,2013-07-26,9,40,35
798697,2015-01-06,8,44,17
105315,2016-05-18,8,6,86
600976,2013-08-11,10,33,93
460961,2015-03-21,3,26,63
672866,2015-06-18,9,37,32
73803,2015-02-03,1,5,9
18787,2014-06-12,1,2,69
859347,2016-02-02,1,48,30


In [3]:
df_test = pd.read_csv('data/test.csv')

df_test = df_test.drop(columns=['id'], axis=1)

df_test.head()

,date,store,item
0,2018-01-01,1,1
1,2018-01-02,1,1
2,2018-01-03,1,1
3,2018-01-04,1,1
4,2018-01-05,1,1


## Data Cleaning

In [4]:
# Null values
df_train.isnull().value_counts()

date   store  item   sales
False  False  False  False    913000
Name: count, dtype: int64

In [5]:
# NA values
df_train.isna().sum()

date     0
store    0
item     0
sales    0
dtype: int64

In [6]:
# Duplicates
df_train.duplicated().sum()

0

In [7]:
df_train.dtypes

date     object
store     int64
item      int64
sales     int64
dtype: object

Since date column is object - convert it to date format

In [8]:
df_train['date'] = pd.to_datetime(df_train['date'], format="%Y-%m-%d")

df_train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [9]:
df_train.dtypes

date     datetime64[ns]
store             int64
item              int64
sales             int64
dtype: object

Make sure the data is sorted as per date

In [10]:
df_train = df_train.sort_values(by=['date', 'store', 'item']).reset_index(drop=True)

df_train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-01,1,2,33
2,2013-01-01,1,3,15
3,2013-01-01,1,4,10
4,2013-01-01,1,5,11


## EDA

In [ ]:
sns.scatterplot(data=df_train, x='date', y='sales')

Sales increases per year - positive trend

In [ ]:
sns.barplot(data=df_train, x='store', y='sales')

In [ ]:
fig = plt.figure(figsize=(20,8))
sns.barplot(data=df_train, x='item', y='sales')

Each store has different demand and each item has a different demand - so each store item pair will have a different demand

In [ ]:
pivoted_sales = df_train.pivot_table(index='date', columns=['store', 'item'], values='sales', aggfunc='sum')

pivoted_sales.head()

## ML Models

### Random Forest

In [ ]:
X_rf = df_train.copy()

In [ ]:
y_rf = X_rf['sales']


Reshaping not required for RF. Convert date to ordinal as RF doesn't understand date type - Add date related features as date is not significant when it comes to regression models

In [ ]:
X_rf['dow'] = X_rf['date'].dt.dayofweek
X_rf['month'] = X_rf['date'].dt.month
X_rf['year'] = X_rf['date'].dt.year


# Converting 'date' column to ordinal to use it as a feature
X_rf['date'] = X_rf['date'].apply(lambda x: x.toordinal())

X_rf.dtypes

In [ ]:
# Dropping the 'sales' column from features
X_rf = X_rf.drop(columns=['sales'])

In [ ]:
# Splitting the data into training and testing sets
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=0.2, random_state=42)

In [ ]:
#Model
rf_model = RandomForestRegressor(n_estimators=100,
                                 max_depth=10,
                                 min_samples_split=2,
                                 min_samples_leaf=1)

In [ ]:
#RF Modeling fitting
rf_model.fit(X_train_rf,y_train_rf)

In [ ]:
#predicting on training set
y_train_pred_rf = rf_model.predict(X_train_rf)

# Calculating Mean Squared Error on training set
mse_rf_train = mean_squared_error(y_train_rf, y_train_pred_rf)
print("Mean Squared Error on Training Set:", mse_rf_train)

In [ ]:
#predicting on testing set
y_test_pred_rf = rf_model.predict(X_test_rf)

# Calculating Mean Squared Error on testing set
mse_rf_test = mean_squared_error(y_test_rf, y_test_pred_rf)
print("Mean Squared Error on Testing Set:", mse_rf_test)

### XGBoost

In [ ]:
X_xgb = df_train.copy()

In [ ]:
y_xgb = X_xgb['sales']

In [ ]:
X_xgb['dow'] = X_xgb['date'].dt.dayofweek
X_xgb['day'] = X_xgb['date'].dt.day
X_xgb['month'] = X_xgb['date'].dt.month
X_xgb['year'] = X_xgb['date'].dt.year

X_xgb.dtypes

In [ ]:
X_xgb = X_xgb.drop(columns=['sales', 'date'])

In [ ]:
# Splitting the data into training and testing sets
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X_xgb, y_xgb, test_size=0.2, random_state=42)

In [ ]:
xg_model = xgb.XGBRegressor(n_estimator=100,learning_rate=0.08, gamma=0, subsample=0.75, colsample_bytree=1, max_depth=7, booster='gbtree')

In [ ]:
xg_model.fit(X_train_xgb,y_train_xgb)

In [ ]:
#predicting on training set
y_train_pred_xgb = xg_model.predict(X_train_xgb)

# Calculating Mean Squared Error on training set
mse_xgb_train = mean_squared_error(y_train_xgb, y_train_pred_xgb)
print("Mean Squared Error on Training Set:", mse_xgb_train)

In [ ]:
#predicting on testing set
y_test_pred_xgb = xg_model.predict(X_test_xgb)

# Calculating Mean Squared Error on testing set
mse_xgb_test = mean_squared_error(y_test_xgb, y_test_pred_xgb)
print("Mean Squared Error on Testing Set:", mse_xgb_test)

### ANN

Need to extract date features, perform scaling and reshaping

In [ ]:
X_ann = df_train.copy()

In [ ]:
# extract y
y_ann = X_ann['sales']

In [ ]:
# scale and reshape y
y_scaler_ann = MinMaxScaler()

y_ann_scaled = y_scaler_ann.fit_transform(y_ann.values.reshape(-1,1))

In [ ]:
X_ann['year'] = X_ann['date'].dt.year
X_ann['month'] = X_ann['date'].dt.month
X_ann['day_of_week'] = X_ann['date'].dt.dayofweek  # Monday=0, Sunday=6
X_ann['day_of_year'] = X_ann['date'].dt.dayofyear
X_ann['week_of_year'] = X_ann['date'].dt.isocalendar().week

X_ann.head()

In [ ]:
X_ann = X_ann.drop(columns=['sales', 'date'])

In [ ]:
X_scaler_ann = MinMaxScaler()

X_ann_scaled = pd.DataFrame(X_scaler_ann.fit_transform(X_ann), columns=X_ann.columns)

In [ ]:
# Splitting the data into training and testing sets
X_train_ann, X_test_ann, y_train_ann, y_test_ann = train_test_split(X_ann_scaled, y_ann_scaled, test_size=0.2, random_state=42)

In [ ]:
## ANN model architecture
ann_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_ann.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)
    ])

In [ ]:
# Compile ANN
ann_model.compile(optimizer=Adam(), loss='mse')

In [ ]:
# Train model
history = ann_model.fit(X_train_ann, y_train_ann, epochs=25, batch_size=32, validation_split=0.1)

In [ ]:
# predicting on training set
y_train_pred_ann = ann_model.predict(X_train_ann)

# mse_ann_train = ann_model.evaluate(X_train_ann, y_train_ann)
mse_ann_train = mean_squared_error(y_train_ann, y_train_pred_ann)

print("Mean Squared Error on Training Set:", mse_ann_train)

In [ ]:
y_train_pred_ann_unscaled = y_scaler_ann.inverse_transform(y_train_pred_ann)

In [ ]:
# predicting on test set
y_test_pred_ann = ann_model.predict(X_test_ann)

# mse_ann_train = ann_model.evaluate(X_train_ann, y_train_ann)
mse_ann_test = mean_squared_error(y_test_ann, y_test_pred_ann)

print("Mean Squared Error on Test Set:", mse_ann_test)

In [ ]:
y_test_pred_ann_unscaled = y_scaler_ann.inverse_transform(y_test_pred_ann)

## Time Series Models

### LSTM

In [ ]:
X_lstm = df_train.copy()

In [ ]:
# extract y
y_lstm = X_lstm['sales']

In [ ]:
X_lstm['day'] = X_lstm['date'].dt.day
X_lstm['month'] = X_lstm['date'].dt.month
X_lstm['dayofweek'] = X_lstm['date'].dt.dayofweek  # 0 = Monday
X_lstm['week'] = X_lstm['date'].dt.isocalendar().week
X_lstm['year'] = X_lstm['date'].dt.year

X_lstm['date'] = X_lstm['date'].apply(lambda x: x.toordinal())

X_lstm.head()

In [ ]:
X_lstm.drop(columns=['sales'], inplace=True)

In [ ]:
# scale and reshape y
y_scaler_lstm = MinMaxScaler()

y_lstm_scaled = y_scaler_lstm.fit_transform(y_lstm.values.reshape(-1,1))

In [ ]:
# scale and reshape X
X_scaler_lstm = MinMaxScaler()

X_lstm_scaled = X_scaler_lstm.fit_transform(X_lstm)

In [ ]:
#  function for creating sequences for LSTM 

def create_sequences(X, y, time_steps=7):
    Xs, ys = [], []
    for i in range(time_steps, len(X)):
        Xs.append(X[i - time_steps:i])
        ys.append(y[i])
    
    Xs = np.array(Xs)

    # If y is 2D (e.g., (n_samples, 1)), preserve its shape
    ys = np.array(ys)
    if len(ys.shape) == 1:
        ys = ys.reshape(-1, 1)

    return Xs, ys


In [ ]:
# Creating sequences for LSTM model
X_lstm_seq, y_lstm_seq = create_sequences(X_lstm_scaled, y_lstm_scaled, time_steps=7)

In [ ]:
# Splitting in test train set
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_lstm_seq, y_lstm_seq, test_size=0.2, random_state=42)

In [ ]:
# Define model architecture
lstm_model = Sequential([
    LSTM(units=50, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dense(units=1)
])

In [ ]:
# compile the LSTM model
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
lstm_model.fit(X_train_lstm, y_train_lstm, epochs=25, batch_size=32, validation_split=0.1)

In [ ]:
# predicting on training set
y_train_pred_lstm = lstm_model.predict(X_train_lstm)

# mse_ann_train = ann_model.evaluate(X_train_ann, y_train_ann)
mse_lstm_train = mean_squared_error(y_train_lstm, y_train_pred_lstm)

print("Mean Squared Error on Training Set:", mse_lstm_train)

In [ ]:
y_train_pred_lstm_unscaled = y_scaler_lstm.inverse_transform(y_train_pred_lstm)

In [ ]:
# predicting on testing set
y_test_pred_lstm = lstm_model.predict(X_test_lstm)

# mse_ann_train = ann_model.evaluate(X_train_ann, y_train_ann)
mse_lstm_test = mean_squared_error(y_test_lstm, y_test_pred_lstm)

print("Mean Squared Error on Test Set:", mse_lstm_test)

In [ ]:
y_test_pred_lstm_unscaled = y_scaler_lstm.inverse_transform(y_test_pred_lstm)

### CNN

########## Best practice:

Split raw data into train/test.

Fit the scaler only on the training set.

Apply the scaler to both train and test.

Then sequence both sets separately.

##### Data - X, y

In [ ]:
X_cnn = df_train.copy()

In [ ]:
# extract y
y_cnn = X_cnn['sales']

In [ ]:
X_cnn['day'] = X_cnn['date'].dt.day
X_cnn['month'] = X_cnn['date'].dt.month
X_cnn['dayofweek'] = X_cnn['date'].dt.dayofweek  # 0 = Monday
X_cnn['week'] = X_cnn['date'].dt.isocalendar().week
X_cnn['year'] = X_cnn['date'].dt.year

X_cnn['date'] = X_cnn['date'].apply(lambda x: x.toordinal())

X_cnn.head()

In [ ]:
X_cnn.drop(columns=['sales'], inplace=True)

##### Splitting

In [ ]:
# Splitting in test train set
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X_cnn, y_cnn, test_size=0.2, random_state=42)

##### Scaling

In [ ]:
# Scaler for y
y_scaler_cnn = MinMaxScaler()

In [ ]:
# Fit Scaler on y train and reshape y train
y_train_cnn_scaled = y_scaler_cnn.fit_transform(y_train_cnn.values.reshape(-1,1))

In [ ]:
# scale and reshape y test
y_test_cnn_scaled = y_scaler_cnn.transform(y_test_cnn.values.reshape(-1,1))

In [ ]:
# Scaler for X
X_scaler_cnn = MinMaxScaler()

In [ ]:
# Fit Scaler on X train
X_train_cnn_scaled = X_scaler_cnn.fit_transform(X_train_cnn)

In [ ]:
# scale X test 
X_test_cnn_scaled = X_scaler_cnn.transform(X_test_cnn)

##### Sequencing (reshaping) X

In [ ]:
X_train_cnn_scaled_seq, y_train_cnn_scaled_seq = create_sequences(X_train_cnn_scaled, y_train_cnn_scaled, time_steps=7)
X_test_cnn_scaled_seq, y_test_cnn_scaled_seq = create_sequences(X_test_cnn_scaled, y_test_cnn_scaled, time_steps=7)

##### Model

In [ ]:
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train_cnn_scaled_seq.shape[1], X_train_cnn_scaled_seq.shape[2])),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)
])

In [ ]:
cnn_model.compile(optimizer='adam', loss='mse')

In [ ]:
cnn_model.fit(X_train_cnn_scaled_seq, y_train_cnn_scaled_seq, epochs=25, batch_size=32, validation_split=0.1)


In [ ]:
loss_cnn_test = cnn_model.evaluate(X_test_cnn_scaled_seq, y_test_cnn_scaled_seq)

print(f"Test Loss: {loss_cnn_test}")


In [ ]:
# predicting on training set
y_test_pred_cnn = lstm_model.predict(X_test_cnn_scaled_seq)

# mse_ann_train = ann_model.evaluate(X_train_ann, y_train_ann)
mse_cnn_test = cnn_model.evaluate(X_test_cnn_scaled_seq, y_test_cnn_scaled_seq)

print("Mean Squared Error on Training Set:", mse_cnn_test)

### BI-LSTM

We are going to use same components that we created for LSTM

In [ ]:
bilstm_model = Sequential([
    Bidirectional(LSTM(units=50, activation='relu'), input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dense(units=1)
])
bilstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
# compile the LSTM model
bilstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
bilstm_model.fit(X_train_lstm, y_train_lstm, epochs=25, batch_size=32, validation_split=0.1)

### ARIMA

In [31]:
X_arima = df_train.copy()

print(X_arima.columns)

Index(['date', 'store', 'item', 'sales'], dtype='object')


Check the skew and see if the data requires adjustment

In [32]:
pre_skew = X_arima['sales'].skew()
pre_kurt = X_arima['sales'].kurt()

print(f'Pre-skew: {pre_skew:.3f}')
print(f'Pre-kurt: {pre_kurt:.3f}')

Pre-skew: 0.867
Pre-kurt: 0.509


Pre-skew = 0.867: This indicates a moderate right (positive) skew, meaning the distribution has a longer tail on the right side - Apply transformation

Pre-kurt = 0.509 < 3: Not many outliers - doesn't need any special handling


In [33]:
#log transformation
X_arima['sales'] = np.log1p(X_arima['sales'])

print(X_arima.columns)


Index(['date', 'store', 'item', 'sales'], dtype='object')


In [34]:
post_skew = X_arima['sales'].skew()
post_kurt = X_arima['sales'].kurt()

print(f'Post-skew: {post_skew:.3f}')
print(f'Post-kurt: {post_kurt:.3f}')

Post-skew: -0.376
Post-kurt: -0.239


In [35]:
y = X_arima['sales']

print(X_arima.columns)

Index(['date', 'store', 'item', 'sales'], dtype='object')


In [16]:
def test_stationarity_for_all(df, freq='D'):
    df['date'] = pd.to_datetime(df['date'])
    
    # Set 'date' as the index
    df.set_index('date', inplace=True)
    
    # Initialize a list to store results
    results = []
    
    # Iterate over all unique store-item combinations
    for (store, item), group in df.groupby(['store', 'item']):
        # Resample sales by the specified frequency (sum or mean depending on your data)
        resampled_sales = group.resample(freq).sum()

        # Apply the ADF test to the resampled sales data
        result = adfuller(resampled_sales['sales'])

        # Store the results
        results.append({
            'store': store,
            'item': item,
            'ADF Test Statistic': result[0],
            'p-value': result[1],
            'Critical Value (1%)': result[4]['1%'],
            'Critical Value (5%)': result[4]['5%'],
            'Critical Value (10%)': result[4]['10%'],
            'Stationary': 'Yes' if result[1] < 0.05 else 'No'
        })
    
    # Convert results into a DataFrame
    results_df = pd.DataFrame(results)
    return results_df

result_stat_arima = test_stationarity_for_all(X_arima)


Most combo are stationary so we will consider overall its stationary

In [36]:
def evaluate_arima_fixed_order(series, arima_order):
    """Evaluate a single ARIMA order on a time series and return MSE."""
    if len(series) < 30:
        return None  # Skip short series

    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    history = list(train)
    predictions = []

    try:
        for t in range(len(test)):
            model = ARIMA(history, order=arima_order)
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model_fit = model.fit()
            yhat = model_fit.forecast(steps=1)[0]
            predictions.append(yhat)
            history.append(test[t])
        error = mean_squared_error(test, predictions)
        return error
    except:
        return None  # Skip if model fails
    

def evaluate_all_combos_fixed_order(df, arima_order):
    """Evaluate fixed ARIMA order across all (store, item) combinations."""
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values('date', inplace=True)
    
    results = []
    grouped = df.groupby(['store', 'item'])

    for (store, item), group in grouped:
        ts = group.groupby('date')['sales'].sum().sort_index()
        mse = evaluate_arima_fixed_order(ts.values, arima_order)
        if mse is not None:
            results.append({'store': store, 'item': item, 'mse': mse})
        else:
            print(f"Skipped {store}-{item} (insufficient data or model failure)")

    results_df = pd.DataFrame(results)
    return results_df

In [38]:
X_arima = df_train.copy()

X_arima['sales'] = np.log1p(X_arima['sales'])

print(X_arima.columns)


Index(['date', 'store', 'item', 'sales'], dtype='object')


In [ ]:
arima_results_711 = evaluate_all_combos_fixed_order(X_arima, (7,1,1))